## Install invana-connector

In [1]:
# for stable version 
# pip install invana

# for installation specific branch
#pip install git+https://github.com/invana/invana@refactor/invana-connectors#egg=invana

# for latest code 
import sys
sys.path.append("../")

In [2]:
def print_list(list_data):
    for data in list_data:
        print("",data)

## 1. Modelling graph

### 1.1 Creating graph instance

In [3]:
from invana import InvanaGraph

gremlin_server_url = "ws://megamind.local:8182/gremlin"

graph = InvanaGraph(gremlin_server_url)

[DEBUG:2023-01-23 08:19:10,629:connector.update_connection_state:61] - GraphConnector state updated to : CONNECTING
[DEBUG:2023-01-23 08:19:10,629:connector._init_connection:99] - create driver connection  
[DEBUG:2023-01-23 08:19:11,634:connector.update_connection_state:61] - GraphConnector state updated to : CONNECTED


### 1.2 Defining Nodes and Relationship Models

In [4]:
from invana.ogm.models import VertexModel, EdgeModel
from invana.ogm.fields import StringProperty, DateTimeProperty, IntegerProperty, FloatProperty, BooleanProperty
from invana.ogm import indexes
from datetime import datetime


class Project(VertexModel):
    graph = graph
    properties = {
        'name': StringProperty(max_length=30, trim_whitespaces=True),
        'description': StringProperty(allow_null=True, min_length=5),
        'rating': FloatProperty(allow_null=True),
        'is_active': BooleanProperty(default=True),
        'created_at': DateTimeProperty(default=lambda: datetime.now())
    }
    indexes = (
        indexes.CompositeIndex("name"),
        indexes.CompositeIndex("created_at")
    )


class Person(VertexModel):
    graph = graph
    properties = {
        'first_name': StringProperty(min_length=5, trim_whitespaces=True),
        'last_name': StringProperty(allow_null=True),
        'username': StringProperty(default="rrmerugu"),
        'member_since': IntegerProperty(),

    }
    indexes = (
        indexes.CompositeIndex("username"),
    )


class Authored(EdgeModel):
    graph = graph
    properties = {
        'created_at': DateTimeProperty(default=lambda: datetime.now()),
        'since': IntegerProperty()
    }
    indexes = (
        indexes.CompositeIndex("created_at")
    )

### 1.3 Installing models on graph

In [ ]:
from invana.helpers.helpers import install_models

is_install_models = True

if is_install_models:
    # i_understand_rollback=True will roll back any transactions in pending.
    install_models(graph, Project, Person, Authored, i_understand_rollback=True)
else:
    print(f"Ignoring models installation because install_models={install_models}")

[DEBUG:2023-01-23 08:19:11,668:events.log_event:55] - Request 9e4289ee-15db-4bbd-80fb-499c28df125c STARTED with query: mgmt = graph.openManagement()
if (mgmt.containsVertexLabel('Project'))
    Project = mgmt.getVertexLabel('Project')
else 
    Project = mgmt.makeVertexLabel('Project').make()
if (mgmt.containsRelationType('name'))
    name = mgmt.getPropertyKey('name')
else 
    name = mgmt.makePropertyKey('name').dataType(String.class).make()
if (mgmt.containsRelationType('description'))
    description = mgmt.getPropertyKey('description')
else 
    description = mgmt.makePropertyKey('description').dataType(String.class).make()
if (mgmt.containsRelationType('rating'))
    rating = mgmt.getPropertyKey('rating')
else 
    rating = mgmt.makePropertyKey('rating').dataType(Float.class).make()
if (mgmt.containsRelationType('is_active'))
    is_active = mgmt.getPropertyKey('is_active')
else 
    is_active = mgmt.makePropertyKey('is_active').dataType(Boolean.class).make()
if (mgmt.containsRel

Rolling back open transactions...


[ERROR:2023-01-23 08:24:31,740:events.log_event:171] - Request a7b0116a-8985-48ce-a3c8-3bf16c55b89b CLIENT_CONNECTION_ERROR with error Cannot connect to host megamind.local:8182 ssl:default [Connect call failed ('192.168.0.40', 8182)] at 2023-01-23 08:24:31.740827
[ERROR:2023-01-23 08:24:31,751:events.log_event:88] - Request a7b0116a-8985-48ce-a3c8-3bf16c55b89b FINISHED with status code : None:None at 2023-01-23 08:24:31.751194; elapsed_time -7.1e-05
[INFO:2023-01-23 08:24:31,758:indexes.create_from_model:13] - Creating index for model - <class '__main__.Project'>
[INFO:2023-01-23 08:24:31,759:indexes.create:33] - Creating index -  <Composite (index_name=CompositeIndexByName; label=None; properties=name />
[DEBUG:2023-01-23 08:24:31,760:events.log_event:55] - Request 34352e8e-a8b0-4c17-8e95-506ea72882d7 STARTED with query: graph.getOpenTransactions().size();; request_options: {};; at 2023-01-23 08:24:31.760466


### 1.4 Execute raw query

In [ ]:
graph.execute_query("g.V().addV('User').property('name','Ravi')", timeout=180)


In [ ]:
response = graph.execute_query("g.V().limit(1).elementMap().toList()", timeout=180)
print(response)

In [ ]:
print(response.data)

In [ ]:
print([d.to_json() for d in response.data])

## 2 CRUD Operations on data

### 2.1 Nodes

#### 2.1.1 Create Nodes 

In [ ]:
person_ravi = Person.objects.get_or_create(first_name="Ravi Raja", last_name="Merugu", member_since=2000)
person_invana = Person.objects.get_or_create(first_name="Ravi Raja", last_name="Merugu", member_since=2000)


project_iengine = Project.objects.create(name="invana engine", rating=5, is_active=True)
project_iconnector = Project.objects.create(name="invana connectors", rating=5, is_active=True)
project_istudio = Project.objects.create(name="invana studio", rating=5, is_active=True)
project_bengine = Project.objects.create(name="browser engine", rating=5, is_active=False)
# print("Created project_iengine", project_iengine)
# print("Created project_iconnector", project_iconnector)
# print("Created project_istudio", project_istudio)
# print("Created project_bengine", project_bengine)
# print("JSON data", project_iengine.to_json())

#### 2.1.2 Read/Search Nodes

In [ ]:
# search
project_search = Project.objects.search().to_list()


## search by ids 
ids = [p.id for p in project_search] 
project_search_has__id = Project.objects.search(has__id=project_search[0].id).to_list()
project_search_has__id_within = Project.objects.search(has__id__within=ids[:2]).to_list()


# search string - eq, neq, startingWith, endingWith, containing, notStartingWith, notEndingWith, notContaining
Project.objects.search(has__name="invana studio").to_list()
Project.objects.search(has__name__eq="invana").to_list()
Project.objects.search(has__name__neq="invana").to_list()
Project.objects.search(has__name__startingWith="invana").to_list()
Project.objects.search(has__name__endingWith="engine").to_list()
Project.objects.search(has__name__containing="eng").to_list()
Project.objects.search(has__name__notStartingWith="invana").to_list()
Project.objects.search(has__name__notEndingWith="engine").to_list()
Project.objects.search(has__name__notContaining="ravi").to_list()


# search int- lt, gt, lte, gte, inside, outside, between
Person.objects.search(has__member_since__lte=3000).to_list()
Person.objects.search(has__member_since__lt=3000).to_list()
Person.objects.search(has__member_since__gte=1999).to_list()
Person.objects.search(has__member_since__gt=1999).to_list()
inside_query = Person.objects.search(has__member_since__inside=(1000, 3000)).to_list()
outside_quuery = Person.objects.search(has__member_since__outside=(1000, 2005)).to_list()
between_query = Person.objects.search(has__member_since__between=(1000, 3000)).to_list()



In [ ]:
# order by 
Project.objects.search().order_by('name').to_list()  # asc order
# Project.objects.search().order_by('-name').to_list()  # desc order <--- this is not working 

In [ ]:
# using range for pagination
Project.objects.search().order_by('name').range(1, 3).to_list()

In [ ]:
from invana.gremlin.paginator import GremlinQuerySetPaginator
import math 

page_size = 5
total_count = Project.objects.search().count()
total_pages = math.ceil(total_count/page_size)
print("total projects count", total_count)
print("page_size", page_size)
print("total_pages", total_pages)

# create queryset and paginator 
queryset = Project.objects.search().order_by("-name")
paginator = GremlinQuerySetPaginator(queryset, page_size)

# run paginator queries
for page_no in range(1, total_pages+1):
    qs = paginator.page(page_no)
    result = qs.to_list()
    print(f"===== Page {page_no} =====", result)

##### 2.1.3 Update Nodes

In [ ]:
Project.objects.search(has__name="invana studio").update(name="Invana Studio")

##### 2.1.4 Delete Nodes 

In [ ]:
Project.objects.delete(has__name__startingWith="browser") # kwargs are same as in search 
Project.objects.search().to_list()

##### 2.1.1.5 get_or_create

In [ ]:
person = Person.objects.get_or_create(first_name="Ravi Raja", last_name="Merugu", member_since=2000)
print(person)
person = Person.objects.get_or_create(first_name="Invana", last_name="Technologies", member_since=2016)
print(person)

##### 2.1.1.6 get_or_none

In [ ]:
Person.objects.get_or_none(first_name="Ravi Raja", last_name="Merugu", member_since=2000)

### 2.2 Relationships

#### 2.2.1 Create Relationship

In [ ]:
person = Person.objects.create(first_name="Ravi Raja", last_name="Merugu", member_since=2000)
project_iengine = Project.objects.create(name="invana engine", rating=5, is_active=True)
project_iconnector = Project.objects.create(name="invana connectors", rating=5, is_active=True)

authored_data = Authored.objects.create(person.id, project_iengine.id, since=2010)
authored_data2 = Authored.objects.create(person.id, project_iconnector.id, since=2020)

#### 2.2.2 Read/Search Relationship

In [ ]:
Authored.objects.search().to_list()
# [d.to_json() for d in data]

In [ ]:
Authored.objects.search(has__since__lt=2020).to_list() # refer search_kwargs from Node search section(2.1.2)

#### 2.2.3 Update Relationship

In [ ]:
Authored.objects.search(has__since__lt=2020).update(since=2050)

#### 2.2.4 Delete Relationship


In [ ]:
Authored.objects.delete()